In [ ]:
#data preprocessing

import os
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

data_dir = 'data'  # Directory containing captured images
processed_dir = 'processed_data'  # Directory to save processed images

# Create a directory for processed images if it does not exist
os.makedirs(processed_dir, exist_ok=True)


def process_images(src_dir, dest_dir):
    for sign in os.listdir(src_dir):
        sign_dir = os.path.join(src_dir, sign)
        if os.path.isdir(sign_dir):
            os.makedirs(os.path.join(dest_dir, sign), exist_ok=True)
            for image_file in os.listdir(sign_dir):
                image_path = os.path.join(sign_dir, image_file)
                image = cv2.imread(image_path)
                if image is not None:
                    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
                        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                        if results.multi_hand_landmarks:
                            for hand_landmarks in results.multi_hand_landmarks:
                                landmarks_array = np.array([[i.x, i.y, i.z] for i in hand_landmarks.landmark],
                                                           dtype=np.float32)
                                if landmarks_array.shape[0] >= 21:
                                    # Convert landmark coordinates to image coordinates
                                    landmarks_image_coords = (landmarks_array[:, :2] * np.array(
                                        [image.shape[1], image.shape[0]])).astype(np.int32)

                                    # Find min and max x, y coordinates
                                    min_x = np.min(landmarks_image_coords[:, 0])
                                    max_x = np.max(landmarks_image_coords[:, 0])
                                    min_y = np.min(landmarks_image_coords[:, 1])
                                    max_y = np.max(landmarks_image_coords[:, 1])

                                    # Check if bounding box coordinates are valid
                                    if min_x < max_x and min_y < max_y:
                                        x, y, width, height = min_x, min_y, max_x - min_x, max_y - min_y
                                        x, y, width, height = x - 10, y - 10, width + 20, height + 20
                                        hand_img = image[y:y + height, x:x + width]
                                        dest_image_path = os.path.join(dest_dir, sign, image_file)
                                        cv2.imwrite(dest_image_path, hand_img)
                                        print(f"Processed: {image_file}")


if __name__ == "__main__":
    process_images(data_dir, processed_dir)
